In [7]:
import requests
import json
import os

# mettre son appID et sa key puis supprimer.
appid = "a5b42571"
key = ""
url = "http://api.adzuna.com/v1/api/jobs/fr/categories?app_id={0}&app_key={1}&&content-type=application/json".format(appid, key)
req1 = requests.get(url).json()

categories = req1["results"]


tags = [] # j'initie cette liste qui récupèrera les tags qui correspondent aux catégories

for cat in categories:
    tags.append(cat["tag"]) # Pour chaque catégorie je récupère son tag

print(tags)

['accounting-finance-jobs', 'it-jobs', 'sales-jobs', 'customer-services-jobs', 'engineering-jobs', 'hr-jobs', 'healthcare-nursing-jobs', 'hospitality-catering-jobs', 'pr-advertising-marketing-jobs', 'logistics-warehouse-jobs', 'teaching-jobs', 'trade-construction-jobs', 'admin-jobs', 'legal-jobs', 'creative-design-jobs', 'graduate-jobs', 'retail-jobs', 'consultancy-jobs', 'manufacturing-jobs', 'scientific-qa-jobs', 'social-work-jobs', 'travel-jobs', 'energy-oil-gas-jobs', 'property-jobs', 'charity-voluntary-jobs', 'domestic-help-cleaning-jobs', 'maintenance-jobs', 'part-time-jobs', 'other-general-jobs', 'unknown']


Connexion à la base de données

In [4]:
from sqlalchemy import create_engine, MetaData, text

username = "user"
password = "password"
hostname = "52.31.100.105"
port = "3306"
database_name = "database"

db_url = f"mysql+mysqlconnector://{username}:{password}@{hostname}:{port}/{database_name}"
metadata = MetaData()
engine = create_engine(db_url)



Création de la base de données

In [6]:
with engine.connect() as connection:
    result = connection.execute(text("CREATE TABLE test2 (nom_poste varchar(255),date_creation datetime,ville varchar(255),pays varchar(255),url varchar(255),nom_entreprise varchar(255),id int,salaire_max int,salaire_min int,salaire int,type_contrat varchar(255),categorie varchar(255),latitude float,longitude float);"))
    

Récupération des données

In [8]:
countries = ["fr"]
results, success, failed = [], [], [] # Création de 3 listes, une pour les résultats, une pour la journalisation des succés et des échecs
for country in countries:
    for page in range(1, 2):
        url = "https://api.adzuna.com/v1/api/jobs/{3}/search/{0}?app_id={1}&app_key={2}".format(page, appid, key, country)
        req = requests.get(url)
        if req.status_code == 200: # Si réponse positive, alors on récupère les données
            try:
                data = req.json()
                results.append(data["results"])
            except:
                failed.append("{} : {}".format(country, page))
                print(req.status_code, " pour la page ", page, "du pays ", country)
            else:
                print(req.status_code, " pour la page ", page, "du pays ", country)
                success.append("{} : {}".format(country, page))

        else :
            print(req.status_code, " pour la page ", page, "du pays ", country)

200  pour la page  1 du pays  fr


Importation des données

In [16]:
with engine.connect() as connection:
    for page in results:
        for annonce in page:
            print(annonce)
            nom_poste = "test"
            date_creation = "2024-02-19"
            ville = annonce['location']['display_name']
            pays = "France"
            url = "petite url"
            nom_entreprise = annonce['company']['display_name']
            id = annonce['id']
            salaire_max = 0
            salaire_min = 0
            salaire = annonce['salary_is_predicted']
            type_contrat = annonce['contract_type']
            categorie = annonce['category']['label']
            latitude = 0
            longitude = 0
            result = connection.execute(text(f"INSERT INTO test2({nom_poste}, {date_creation}, {ville}, {pays}, {url}, {nom_entreprise}, {id}, {salaire_max}, {salaire_min}, {salaire}, {type_contrat}, {categorie}, {latitude}, {longitude});"))

    resultat = connection.execute(text("SELECT * FROM Test2;"))
    print(resultat.fetchall())

{'redirect_url': 'https://www.adzuna.fr/details/4574741466?utm_medium=api&utm_source=a5b42571', 'contract_type': 'contract', 'created': '2024-02-19T20:04:01Z', 'title': 'Business Developer indépendant(e) (H/F)', 'salary_is_predicted': '0', 'company': {'__CLASS__': 'Adzuna::API::Response::Company', 'display_name': 'Predictis'}, 'location': {'__CLASS__': 'Adzuna::API::Response::Location', 'display_name': 'Montgeron, Evry', 'area': ['France', 'Ile-de-France', 'Essonne', 'Evry', 'Montgeron']}, 'id': '4574741466', 'latitude': 48.70662, 'description': 'Envie de changer de vie ? La gestion de patrimoine vous intéresse ? Business developer indépendant. Bien plus qu’une carrière, une aventure unique  Partage, digital et formation, sont les bases du réseau Predictis qui développe des partenariats avec des centaines de conseillers répartis sur toute la France. PREDICTIS, 1ER COURTIER EN EPARGNE-VIE Créé en 1997, et certifié ISO 9001, Predictis est spécialisé en solutions patrimoniales, épargne re

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '2024-02-19, Montgeron, Evry, France, petite url, Predictis, 4574741466, 0, 0, 0,' at line 1
[SQL: INSERT INTO test2(test, 2024-02-19, Montgeron, Evry, France, petite url, Predictis, 4574741466, 0, 0, 0, contract, Emplois Vente, 0, 0);]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
with engine.connect() as connection:
    for page in results:
        for annonce in page:
            print(annonce)
            nom_poste = annonce['title']
            date_creation = "2024-02-19"
            ville = annonce['location']['display_name']
            pays = annonce['location']['area'][0]
            url = annonce['redirect_url']
            nom_entreprise = annonce['company']['display_name']
            id = annonce['id']
            salaire_max = 0
            salaire_min = 0
            salaire = annonce['salary_is_predicted']
            type_contrat = annonce['contract_type']
            categorie = annonce['category']['label']
            latitude = annonce['latitude']
            longitude = annonce['longitude']
            result = connection.execute(text(f"INSERT INTO test2({nom_poste}, {date_creation}, {ville}, {pays}, {url}, {nom_entreprise}, {id}, {salaire_max}, {salaire_min}, {salaire}, {type_contrat}, {categorie}, {latitude}, {longitude});"))

    resultat = connection.execute(text("SELECT * FROM Test2;"))
    print(resultat.fetchall())